<a href="https://colab.research.google.com/github/marendtz/News_NER/blob/master/XLMR_finetune_pan_and_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparations**

In [44]:
# for debugging: clean up model
import shutil
try:
  shutil.rmtree('/content/maren-hugg/xlm-roberta-base-finetuned-panx-en-news')
except:
  print("no such folder")

no such folder


In [2]:
# get github access token for collab
import getpass
token = getpass.getpass()

··········


In [3]:
# clean workdir
%cd /content/
%rm -r /content/*
%ls -la
%pwd

/content
total 16
drwxr-xr-x 1 root root 4096 Apr 30 12:15 ./
drwxr-xr-x 1 root root 4096 Apr 30 11:52 ../
drwxr-xr-x 4 root root 4096 Apr 27 13:34 .config/


'/content'

In [4]:
# clone github repo
!git clone https://{token}@github.com/marendtz/News_NER.git

Cloning into 'News_NER'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 26 (delta 11), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), 171.83 KiB | 2.86 MiB/s, done.


In [5]:
# install dependencies
%pip install -r /content/News_NER/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 

In [6]:
# import packages
from collections import defaultdict
import pandas as pd
import numpy as np
from datasets import load_dataset_builder, get_dataset_config_names, load_dataset, DatasetDict

from transformers import AutoTokenizer, TrainingArguments, DataCollatorForTokenClassification, Trainer
import nltk.data
nltk.download('punkt')

import torch.nn as nn
from transformers import XLMRobertaConfig, AutoConfig, XLMRobertaForTokenClassification
from transformers.modeling_outputs import TokenClassifierOutput

from huggingface_hub import notebook_login
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
from seqeval.scheme import IOB2


from torch.nn.functional import cross_entropy
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Get and Analyze dataset from Huggingface**

In [7]:
# login to HugginFace to enable upload of model
notebook_login()

In [8]:
# define model

# get info from dataset later used for training our model
# load dataset to get the tags used, so we later take the same...

ds = load_dataset("maren-hugg/news_ner_dict")
tags = ds["train"].features["ner_tags"].feature
print(tags)

# generate function to convert name and id of tags
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
print(index2tag)
print(tag2index)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/maren-hugg___parquet/maren-hugg--news_ner_dict-912bfa5b6de7e043/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)
{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


**Prepare input data for following training of the model XLMRobertaForTokenClassification (with our config=labels, device, ...)**

In [15]:
# prepare dataset
ds_custom = ds
print("Example element in Custom Dataset:")
ds_custom_element = ds["train"][1]
for key, value in ds_custom_element.items():
    print(f"{key}: {value}")

Example element in Custom Dataset:
ner_tags: [0, 0, 0, 0, 1, 0, 0, 0, 0, 5, 0]
tokens: ['Hello', 'my', 'Name', 'is', 'Maren', 'and', "I'm", 'testing', 'from', 'Mannheim', '.']


In [16]:
# show info about tag feature --> we already defined the varibale tags earlier
# --> DatasetDict holds information about all features used (like infos on columns in dataframe)
print('----------features----------------')
print(ds_custom["train"].features)
print('----------feature ner_tags----------------')
print(ds_custom["train"].features["ner_tags"].feature)


----------features----------------
{'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}
----------feature ner_tags----------------
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


In [18]:
# examples
print(ds_custom["train"][1])
print(ds_custom["train"][1]["tokens"])
print(ds_custom["train"][1]["ner_tags"])

{'ner_tags': [0, 0, 0, 0, 1, 0, 0, 0, 0, 5, 0], 'tokens': ['Hello', 'my', 'Name', 'is', 'Maren', 'and', "I'm", 'testing', 'from', 'Mannheim', '.']}
['Hello', 'my', 'Name', 'is', 'Maren', 'and', "I'm", 'testing', 'from', 'Mannheim', '.']
[0, 0, 0, 0, 1, 0, 0, 0, 0, 5, 0]


**Set tokenizer**

In [34]:
# setup for tokenization
xlmr_model_name = "maren-hugg/xlm-roberta-base-finetuned-panx-en"
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

**Define helper function for data preparation**

In [35]:
# the library datasets provides a fast way to tokenize dataset objects with the map() operation.
# the returned input ids need to be augmented with the attention mask and the label ids, that encode the information about with token is associated with each NER tag

def tokenize_and_align_labels(examples):
  tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)  # sequence is already split in words
  labels = []
  for idx, label in enumerate(examples["ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=idx) # get word ids to associate subwords
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100) # -100 is chose, since the PyTorch cross-entropy loss class torch.nn.CrossEntropyLoss has an attribute ignore_index, whose value is -100, which is therefor ignored during training
      else:
        label_ids.append(label[word_idx])
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

def encode_ds_custom_dataset(corpus):
  return corpus.map(tokenize_and_align_labels, batched=True,remove_columns=['ner_tags', 'tokens'])



In [36]:
# encode dataset
ds_custom_encoded = encode_ds_custom_dataset(ds_custom)
print("Encoded example:")
print(ds_custom_encoded["train"][1])

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Encoded example:
{'input_ids': [0, 35378, 759, 15757, 83, 1784, 33, 136, 87, 25, 39, 134234, 1295, 14912, 9491, 6, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 1, -100, 0, 0, -100, -100, 0, 0, 5, -100, 0, -100, -100]}


**Define performance measures**

In [37]:
# common approach: results for precision, recall, F1-score
# in NER: all words of an entity need to be predicted correctly in order for a prediction to be counted as correct
# libary seqeval: expects predictions and labels as lists of lists 


# during training we need to convert the outputs of the model into such a list that sequeval expects
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []

  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      # Ignore label IDs = -100
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

# helper function for calculation of F1-score for validation set
def compute_metrics(eval_pred):
  y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)
  return {"f1": f1_score(y_true, y_pred),
          "precision": precision_score(y_true, y_pred),
          "recall": recall_score(y_true, y_pred),
          "accuracy": f1_score(y_true, y_pred),
          "classification_report": classification_report(y_true, y_pred, mode='strict', scheme=IOB2)
          }


**Finetuning XLM-RoBERTa and Upload to Huggingface**

In [29]:
# hide_output

import torch

# passing keyword arguments to the from_pretrained() method overrides default values
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,num_labels=tags.num_classes, id2label=index2tag, label2id=tag2index)

# define "device"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [51]:
# docu: https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments
num_epochs = 10
batch_size = 2
logging_steps = len(ds_custom_encoded["train"]) // batch_size 
model_name = f"{xlmr_model_name}-news"
training_args = TrainingArguments(output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="steps", eval_steps=batch_size, # Number of update steps between two evaluations if evaluation_strategy="steps".
                                  save_strategy="steps", save_steps=1e6, 
                                  weight_decay=0.01,
                                  disable_tqdm=False, 
                                  logging_strategy="steps", logging_steps=batch_size,  # Number of update steps between two logs if logging_strategy="steps".
                                  push_to_hub=True)

In [53]:
# this model needs to be crated at huggingface:
f"{xlmr_model_name}-news"

'maren-hugg/xlm-roberta-base-finetuned-panx-en-news'

In [54]:
# pad each input sequence to the largest sequence length in a batch
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [55]:
# init method, to avoid initializing a new model for every Trainer (it loads the untrained model and is called at the beginning of the train() call)
def model_init():
  return (XLMRobertaForTokenClassification
          .from_pretrained(xlmr_model_name, config=xlmr_config)
          .to(device))

In [56]:
trainer = Trainer(model_init=model_init, args=training_args,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  train_dataset=ds_custom_encoded["train"],
                  eval_dataset=ds_custom_encoded["validation"],
                  tokenizer=xlmr_tokenizer)

Cloning https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en-news into local empty directory.


In [57]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy,Classification Report
2,0.063800,0.145141,0.666667,1.000000,0.500000,0.666667,precision recall f1-score support ORG 1.00 0.50 0.67 2 micro avg 1.00 0.50 0.67 2 macro avg 1.00 0.50 0.67 2 weighted avg 1.00 0.50 0.67 2
4,0.004500,0.117295,0.666667,1.000000,0.500000,0.666667,precision recall f1-score support ORG 1.00 0.50 0.67 2 micro avg 1.00 0.50 0.67 2 macro avg 1.00 0.50 0.67 2 weighted avg 1.00 0.50 0.67 2
6,0.001700,0.106300,0.666667,1.000000,0.500000,0.666667,precision recall f1-score support ORG 1.00 0.50 0.67 2 micro avg 1.00 0.50 0.67 2 macro avg 1.00 0.50 0.67 2 weighted avg 1.00 0.50 0.67 2
8,0.001300,0.101198,0.666667,1.000000,0.500000,0.666667,precision recall f1-score support ORG 1.00 0.50 0.67 2 micro avg 1.00 0.50 0.67 2 macro avg 1.00 0.50 0.67 2 weighted avg 1.00 0.50 0.67 2
10,0.001400,0.098888,0.666667,1.000000,0.500000,0.666667,precision recall f1-score support ORG 1.00 0.50 0.67 2 micro avg 1.00 0.50 0.67 2 macro avg 1.00 0.50 0.67 2 weighted avg 1.00 0.50 0.67 2


TrainOutput(global_step=10, training_loss=0.014516227110289038, metrics={'train_runtime': 0.6846, 'train_samples_per_second': 29.214, 'train_steps_per_second': 14.607, 'total_flos': 183732587280.0, 'train_loss': 0.014516227110289038, 'epoch': 10.0})

In [58]:
trainer.push_to_hub(commit_message="Training completed!")

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


Upload file pytorch_model.bin:   0%|          | 1.00/1.03G [00:00<?, ?B/s]

Upload file tokenizer.json:   0%|          | 1.00/16.3M [00:00<?, ?B/s]

Upload file runs/Apr30_12-36-52_b6343ff52dc8/1682858301.3066504/events.out.tfevents.1682858301.b6343ff52dc8.49…

Upload file runs/Apr30_12-36-52_b6343ff52dc8/events.out.tfevents.1682858301.b6343ff52dc8.4915.4:   0%|        …

Upload file training_args.bin:   0%|          | 1.00/3.56k [00:00<?, ?B/s]

To https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en-news
   6ceb702..f453fed  main -> main

   6ceb702..f453fed  main -> main

To https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en-news
   f453fed..138def8  main -> main

   f453fed..138def8  main -> main



'https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en-news/commit/f453fed430e4fe011db8d354a8d2626d51ad1423'

In [59]:
# info about trainer log history
metrics_train=trainer.state.log_history
for item in metrics_train:
  print(item)

{'loss': 0.0638, 'learning_rate': 4e-05, 'epoch': 2.0, 'step': 2}
{'eval_loss': 0.14514127373695374, 'eval_f1': 0.6666666666666666, 'eval_precision': 1.0, 'eval_recall': 0.5, 'eval_accuracy': 0.6666666666666666, 'eval_classification_report': '              precision    recall  f1-score   support\n\n         ORG       1.00      0.50      0.67         2\n\n   micro avg       1.00      0.50      0.67         2\n   macro avg       1.00      0.50      0.67         2\nweighted avg       1.00      0.50      0.67         2\n', 'eval_runtime': 0.0177, 'eval_samples_per_second': 56.34, 'eval_steps_per_second': 56.34, 'epoch': 2.0, 'step': 2}
{'loss': 0.0045, 'learning_rate': 3e-05, 'epoch': 4.0, 'step': 4}
{'eval_loss': 0.11729458719491959, 'eval_f1': 0.6666666666666666, 'eval_precision': 1.0, 'eval_recall': 0.5, 'eval_accuracy': 0.6666666666666666, 'eval_classification_report': '              precision    recall  f1-score   support\n\n         ORG       1.00      0.50      0.67         2\n\n   